In [1]:
import os

import cv2
import gdown
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tt
import json
import pandas as pd
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchvision.utils import save_image
from tqdm.notebook import tqdm
#from google.colab import drive, files
import nbimporter
import copy
import time

from functions import *
root = '.'

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model
from scipy.linalg import sqrtm

def calculate_fid(image_path1, image_path2):
    def calculate_activation_statistics(images, model):
        print(images.shape)
        act1 = model.predict(images)
        act1 = act1.reshape(act1.shape[0], -1)
        print(act1.shape)
        mu, sigma = act1.mean(axis=0), np.cov(act1, rowvar=False)
        return mu, sigma

    def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
        diff = mu1 - mu2
        # Calculate the matrix square root of the product of sigma1 and sigma2
        covmean = sqrtm(sigma1.dot(sigma2))
        if np.iscomplexobj(covmean):
            covmean = covmean.real
        # Calculate FID
        fid = np.dot(diff, diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * np.trace(covmean)
        return fid

    # Load InceptionV3 model without top classification layer
    inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(75, 75, 3))
    inception_model = Model(inputs=inception_model.input, outputs=inception_model.layers[-2].output)

    # Load and preprocess images
    images1 = preprocess_input(load_and_preprocess_images(image_path1, (75, 75)))
    images2 = preprocess_input(load_and_preprocess_images(image_path2, (75, 75)))

    
    mu1, sigma1 = calculate_activation_statistics(images1, inception_model)
    mu2, sigma2 = calculate_activation_statistics(images2, inception_model)

    # Calculate FID
    fid = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)
    return fid

def load_and_preprocess_images(image_path, target_size):
    image_paths = [os.path.join(image_path, fname) for fname in os.listdir(image_path)]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=target_size) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img).astype('float32') for img in images]
    return np.array(images)

In [3]:
def gen_photos(generator, model_name, device, latent_size, generator_path, photos_number):
    current_directory = os.getcwd()
    generator_path = 'tmp'
    # Sprawdź, czy folder generator_path istnieje, jeśli nie, utwórz go
    root = os.path.join(current_directory, generator_path)
    if not os.path.exists(root):
        os.makedirs(root)
    for x in range(photos_number):
        if model_name == 'lsgan':
            fixed_latent = torch.randn(1, latent_size, device=device)
        else:
            fixed_latent = torch.randn(1, latent_size, 1, 1, device=device)
        fake_images = generator(fixed_latent)

        fake_fname = f'{x}.png'

        # Utwórz pełną ścieżkę do pliku docelowego
        fake_image_path = os.path.join(root, fake_fname)

        try:
            save_image(denorm(fake_images, stats), fake_image_path, nrow=8)
        except Exception as e:
            print(f'Błąd podczas zapisywania pliku: {e}')


In [ ]:
all_fid = []

# Epochs FID

In [4]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
model_name = 'basic'


Read parameters


In [ ]:
print('Start')
epochs_fid = {}
epochsss = [5, 15, 30, 50, 100]
exp_title = 'test_epochs'
photos_number = 15747
del epochs
for exp_index, epochs in enumerate(epochsss):
    generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
    generator = get_generator(device, latent_size, model_name)
    generator.load_state_dict(torch.load(f'{generator_path}'))
    gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
    print("OK")
    epochsss = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
          r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
all_fid.append(epochsss)

Start
OK


100%|██████████| 1/1 [00:04<00:00,  4.56s/it]


# LR FID

In [ ]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
model_name = 'basic'



In [ ]:
print('Start')
lrs = [0.00005, 0.0001, 0.0002, 0.0004, 0.0006]
exp_title = 'test_lr'
lrs = {}
del lr
for exp_index, lr in enumerate(lrs):
    generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
    generator = get_generator(device, latent_size, model_name)
    generator.load_state_dict(torch.load(f'{generator_path}'))
    gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
    print("OK")
    lrs = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
          r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
all_fid.append(lrs)

# Batch size FID

In [ ]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
model_name = 'basic'

In [ ]:
print('Start')
batch_sizes = [8, 32, 128, 512, 1024]
exp_title = 'test_batch'
batches = {}
del batch_size
for exp_index, batch_size in enumerate(batch_sizes):
    generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
    generator = get_generator(device, latent_size, model_name)
    generator.load_state_dict(torch.load(f'{generator_path}'))
    gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
    print("OK")
    batches = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
          r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
all_fid.append(batches)

# Latent FID

In [ ]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
model_name = 'basic'


In [ ]:
print('Start')
latent_sizes = [8, 32, 128, 512, 1024]
exp_title = 'test_latent'
del latent_size
latents = {}
for exp_index, latent_size in enumerate(latent_sizes):   
    generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
    generator = get_generator(device, latent_size, model_name)
    generator.load_state_dict(torch.load(f'{generator_path}'))
    gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
    print("OK")
    latents = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
          r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
all_fid.append(latents)

# Optimizer FID


In [ ]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
latent_size = 128
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
model_name = 'basic'


In [ ]:
print('Start')
optimizers_list = [torch.optim.RAdam, torch.optim.NAdam, torch.optim.AdamW, torch.optim.Adam]
exp_title = 'test_optimizer'
optimizerXXX = {}
for exp_index, optimizer_test in enumerate(optimizers_list):   
    generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
    generator = get_generator(device, latent_size, model_name)
    generator.load_state_dict(torch.load(f'{generator_path}'))
    gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
    print("OK")
    optimizerXXX = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
          r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
all_fid.append(optimizerXXX)

# Loss FID

In [ ]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
latent_size = 128
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
model_name = 'basic'


In [ ]:
print('Start')
loss_list = [nn.MSELoss, nn.BCELoss]
exp_title = 'test_loss'
lossesXXX = {}
for exp_index, lost_test in enumerate(loss_list):   
    generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
    generator = get_generator(device, latent_size, model_name)
    generator.load_state_dict(torch.load(f'{generator_path}'))
    gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
    print("OK")
    lossesXXX = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
          r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
all_fid.append(lossesXXX)

# Model FID

In [ ]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
latent_size = 128
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)

In [ ]:
print('Start')
loss_list = [nn.MSELoss, nn.BCELoss]
model_names = ['lsgan', 'basic']
weights_types = [True, False]
exp_title = 'test_models'
exp_index = 0
modelsXXX = {}
for loss_test in loss_list:
    for model_name in model_names:
        for weight_type in weights_types:
            if loss_test == nn.BCELoss and model_name == 'lsgan':
                continue


            generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
            generator = get_generator(device, latent_size, model_name)
            generator.load_state_dict(torch.load(f'{generator_path}'))
            gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
            print("OK")
            modelsXXX = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
                  r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
            exp_index += 1
all_fid.append(modelsXXX)

# Architekra FID

In [ ]:
print("Read parameters")
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
DATA_DIR = 'cats'
image_size = 64
latent_size = 128
train_ds = ImageFolder(DATA_DIR,
                       transform=tt.Compose([tt.Resize(image_size),
                                             tt.CenterCrop(image_size),
                                             tt.ToTensor(),
                                             tt.Normalize(*stats)]))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 55
lr = 0.0002
batch_size = 128
latent_size = 128
DATA_DIR = 'cats'

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
train_dl = DeviceDataLoader(train_dl, device)
model_name = 'basic'


In [ ]:
print('Start')
activations_gen = [nn.ReLU(True), nn.LeakyReLU(0.2, inplace=True), nn.LeakyReLU(0.5, inplace=True)]
activations_dis = [nn.LeakyReLU(0.2, inplace=True), nn.ReLU(True), nn.LeakyReLU(0.5, inplace=True)]
exp_title = 'test_arch_trash'
exp_index = 0
architer = {}
for activation_gen in enumerate(activations_gen): 
    for activation_dis in enumerate(activations_dis):
        generator_path = 'generator_' + exp_title + '_' + str(exp_index) + '_' + model_name
        generator = get_generator(device, latent_size, model_name)
        generator.load_state_dict(torch.load(f'{generator_path}'))
        gen_photos(generator, model_name, device, latent_size, generator_path, photos_number)
        print("OK")
        architer = {exp_index: calculate_fid(r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\cats_1000', 
              r'C:\Users\karol\Downloads\lista-1-KunickiKarol-main\tmp')}
        exp_index += 1
all_fid.append(architer)